In [13]:
import requests
import BeautifulSoup as soup
import time
import pandas as pd

In [4]:
# First page to crawl
url = 'https://dogvacay.com/dog-boarding--tx--austin'

In [5]:
# TODO: check if the page has multiple pages (most do, and use the url below to get the 2nd, 3rd, etc page data too)
# https://dogvacay.com/dog-boarding--tx--austin?p=2

In [6]:
# TODO: set up a loop until you've checked all pages

In [7]:
# TODO: set up a loop over other cities too

In [8]:
# This gets the page from the web, and also waits one second after you do
# The one second wait is to make sure you don't accidentally make too many requests which is bad netiquette
r = requests.get(url)
time.sleep(1)

In [9]:
# Make sure you got a 200 response, otherwise it indicates an error
r.status_code

200

In [10]:
# Convert the text to a beautiful soup object
s = soup.BeautifulSoup(r.text)

In [11]:
# Remeber the "inspect element" trick I showed you in Chrome?  Use that to find the data you want
sitters = s.findAll('div', {'class': 'card-content'})
len(sitters)

15

In [14]:
# Data extraction phase
# TODO: This is a work in progress.  You should have a go at getting it working.
# TODO: Pick which important fields you want to extract, find them, build the data frame

sitter = sitters[0]
times = []
fees = []
city = []
for sitter in sitters:
    response_time = sitter.findAll('div', {'class': 'icon-wrapper'})
    fee = sitter.findAll('div', {'class': 'price price--primary u-text-center'})
    fees.append(fee)
    city.append('Austin')

df = pd.DataFrame({'city': city, 'fee': fees})
df

,city,fee
0,Austin,"[<div class=""price price--primary u-text-cente..."
1,Austin,"[<div class=""price price--primary u-text-cente..."
2,Austin,"[<div class=""price price--primary u-text-cente..."
3,Austin,"[<div class=""price price--primary u-text-cente..."
4,Austin,"[<div class=""price price--primary u-text-cente..."
5,Austin,"[<div class=""price price--primary u-text-cente..."
6,Austin,"[<div class=""price price--primary u-text-cente..."
7,Austin,"[<div class=""price price--primary u-text-cente..."
8,Austin,"[<div class=""price price--primary u-text-cente..."
9,Austin,"[<div class=""price price--primary u-text-cente..."
